In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
file = ['dseb63_application_test','dseb63_application_train','dseb63_bureau',\
        'dseb63_bureau_balance', 'dseb63_credit_card_balance','dseb63_installments_payments',\
        'dseb63_POS_CASH_balance','dseb63_previous_application']
link = 'D:\\Năm 3 - HK1\\Visualize\\Ex\\final_project\\dseb63_final_project_DP_dataset\\dseb63_final_project_DP_dataset\\'
df = pd.read_csv(link + file[1]+'.csv',index_col='Unnamed: 0')
test_set = pd.read_csv(link + file[0]+'.csv',index_col='Unnamed: 0')
submit = pd.read_csv('D:\\Năm 3 - HK1\\Visualize\\Ex\\final_project\\dseb63_sample_submission.csv')


# DROP

In [3]:

#null high
col_drop =  (df.isnull().sum()/df.shape[0]).sort_values(ascending=False).index[:50].tolist()
#flag doc
starting_idx = df.columns.get_loc("FLAG_DOCUMENT_2")
ending_idx = df.columns.get_loc("FLAG_DOCUMENT_21")+1
for i in range(starting_idx,ending_idx):
    col_drop.append(df.columns[i])
col_drop.remove('FLAG_DOCUMENT_3')
#social
for i in ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE']:
    col_drop.append(i)
#enquiries freq
for i in ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_QRT']:
    col_drop.append(i)
#contact
for i in ['FLAG_MOBIL','FLAG_CONT_MOBILE','FLAG_EMAIL','FLAG_EMP_PHONE']:
    col_drop.append(i)

# region
for i in ['REGION_RATING_CLIENT','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION']:
    col_drop.append(i)
# finance
for i in ['AMT_GOODS_PRICE']:
    col_drop.append(i)
#family
col_drop.append('CNT_FAM_MEMBERS')

#cate
for i in ['NAME_CONTRACT_TYPE','NAME_HOUSING_TYPE','NAME_TYPE_SUITE']:
    col_drop.append(i)
for i in ['OCCUPATION_TYPE','ORGANIZATION_TYPE']:
    col_drop.append(i)
for i in ['WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START']:
    col_drop.append(i)

col_drop.append('SK_ID_CURR')

In [4]:
new = df.drop(col_drop,axis = 1)
# new.describe()
test_set = test_set.drop(col_drop,axis = 1)
new.dropna(subset='TARGET',inplace=True)

In [5]:
new.loc[new['CODE_GENDER']=='XNA'] = np.nan
new.loc[new['NAME_FAMILY_STATUS']=='Unknown'] = np.nan

In [6]:
def turn_to_numeric(df,feature):
    label = df[feature].value_counts().index.tolist()
    df[feature] = df[feature].apply(lambda x: check(x,label))
    return df
def check(val,list):
    for i in range(len(list)):
        if val == list[i]:
            return i
    return np.nan 

In [7]:
cols = ['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS',]
for col in cols:
    turn_to_numeric(new,col)
    turn_to_numeric(test_set,col)

In [8]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
data = imputer.fit_transform(new)

In [10]:
# new2[cols] = new2[cols].apply(lambda x: round(x)) 
new2= pd.DataFrame(data,columns=new.columns)
new2.to_csv('D:\\Năm 3 - HK1\\Visualize\\Ex\\final_project\\aftereda.csv')

In [12]:
from sklearn.preprocessing import MinMaxScaler
# Make the model with the specified regularization parameter
scaler = MinMaxScaler(feature_range = (0, 1))

x_train = new2.iloc[:,1:]
y_train = new2.iloc[:,0]

scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(test_set)
print('Training data shape: ', x_train.shape)
print('Testing data shape: ', x_test.shape)

Training data shape:  (246009, 28)
Testing data shape:  (61502, 28)


In [16]:
from sklearn.linear_model import LogisticRegression
# Train on the training data
reg = LogisticRegression()

y_train = y_train.apply(lambda x: round(x))
reg.fit(x_train, y_train)
y_pred = reg.predict_proba(x_test)[:, 1]
submit['TARGET'] = y_pred

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# summit.to_csv('D:\\Năm 3 - HK1\\Visualize\\Ex\\final_project\\adsubmit2.csv')

In [17]:
test_set.describe()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR
count,61502.000000,61502.000000,61502.000000,61502.000000,6.150200e+04,6.150200e+04,61500.000000,61502.000000,61502.000000,61502.000000,...,61502.000000,61502.000000,61502.000000,61371.000000,49361.000000,61286.000000,61502.000000,61502.000000,53194.000000,53194.000000
mean,0.340005,0.337583,0.309470,0.420344,1.701541e+05,5.965660e+05,27061.917220,0.803096,0.345729,0.751374,...,0.077412,0.231700,0.181181,0.514422,0.510129,0.099794,-963.583298,0.710579,0.270388,1.895364
std,0.473749,0.472889,0.462279,0.727919,4.872706e+05,4.000463e+05,14425.973334,0.970965,0.609377,1.189246,...,0.267246,0.421922,0.385171,0.191089,0.194873,0.357433,823.498386,0.453498,0.932013,1.865405
min,0.000000,0.000000,0.000000,0.000000,2.610000e+04,4.500000e+04,1980.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000006,0.000527,0.000000,-4128.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.125000e+05,2.700000e+05,16582.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.393396,0.368969,0.000000,-1569.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.485000e+05,5.099220e+05,24903.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.566311,0.535276,0.000000,-760.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,2.025000e+05,8.086500e+05,34465.500000,2.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.663141,0.669057,0.000000,-278.000000,1.000000,0.000000,3.000000
max,2.000000,1.000000,1.000000,12.000000,1.170000e+08,4.050000e+06,230161.500000,7.000000,4.000000,5.000000,...,1.000000,1.000000,1.000000,0.855000,0.887664,6.000000,0.000000,1.000000,22.000000,23.000000


In [1]:
a= None
a == None

True